# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Homecage-odor-exposure" data-toc-modified-id="Homecage-odor-exposure-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Homecage odor exposure</a></div><div class="lev2 toc-item"><a href="#Import-data" data-toc-modified-id="Import-data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import data</a></div><div class="lev3 toc-item"><a href="#Import-behavior" data-toc-modified-id="Import-behavior-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Import behavior</a></div><div class="lev3 toc-item"><a href="#Import-calcium-imaging-data" data-toc-modified-id="Import-calcium-imaging-data-112"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Import calcium imaging data</a></div><div class="lev2 toc-item"><a href="#Save-data" data-toc-modified-id="Save-data-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Save data</a></div><div class="lev1 toc-item"><a href="#Elevated-plus-maze" data-toc-modified-id="Elevated-plus-maze-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Elevated-plus maze</a></div><div class="lev2 toc-item"><a href="#Import-data" data-toc-modified-id="Import-data-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import data</a></div><div class="lev3 toc-item"><a href="#Import-behavioral-data" data-toc-modified-id="Import-behavioral-data-211"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Import behavioral data</a></div><div class="lev3 toc-item"><a href="#Downsample-data" data-toc-modified-id="Downsample-data-212"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Downsample data</a></div><div class="lev3 toc-item"><a href="#Import-calcium-imaging-data" data-toc-modified-id="Import-calcium-imaging-data-213"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Import calcium imaging data</a></div><div class="lev3 toc-item"><a href="#Downsample-data-(if-necessary)" data-toc-modified-id="Downsample-data-(if-necessary)-214"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Downsample data (if necessary)</a></div><div class="lev3 toc-item"><a href="#Save-data" data-toc-modified-id="Save-data-215"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Save data</a></div><div class="lev1 toc-item"><a href="#Headfixed-exposure" data-toc-modified-id="Headfixed-exposure-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Headfixed exposure</a></div><div class="lev2 toc-item"><a href="#Create-behavioral-data" data-toc-modified-id="Create-behavioral-data-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create behavioral data</a></div><div class="lev2 toc-item"><a href="#Import-neural-data" data-toc-modified-id="Import-neural-data-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Import neural data</a></div><div class="lev3 toc-item"><a href="#Save-data" data-toc-modified-id="Save-data-321"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Save data</a></div>

In [1]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py as h5
import os
import time
import glob
import multiprocessing as mp
import custom
import pdb

---
<a id='homecage'></a>
# Homecage odor exposure

## Import data

In [ ]:
# Parameters
bin_size = 200
n_cores = 8

### Import behavior

In [ ]:
# Define import function
def import_behav(filename):
    subj, epoch = os.path.splitext(os.path.basename(filename))[0].split('_')
    data = custom.etho_extract(filename)
    data.index = data.index * 1000
    
    return (subj, epoch), data

In [ ]:
# Import data
# This will lock up often. Not sure why...
# Lowered number of processes in pool and seems to work better... nope
# Keep Excel files closed seems to do the trick... nah
# Define pool AFTER defining all functions is the fix:
#     https://stackoverflow.com/questions/2782961/yet-another-confusion-with-multiprocessing-error-module-object-has-no-attribu
#     https://stackoverflow.com/questions/18947876/using-python-multiprocessing-pool-in-the-terminal-and-in-code-modules-for-django

behav_files = glob.glob(behav_source)
p = mp.Pool(processes=n_cores)
exps, behav_import = zip(*p.map(import_behav, behav_files))

In [ ]:
# Create dataframe from all animals

ts = np.arange(0, 300000, bin_size)
subjs = np.unique([x for x, _ in exps])

dfs = []
for subj in subjs:
    epoch_key, subj_exps = zip(*[(y, behav_import[n]) for n, (x, y) in enumerate(exps) if x == subj])

    # Downsample data and data frame for subject (with each epoch)
    subj_df = pd.DataFrame()
    for exp, epoch_name in zip(subj_exps, epoch_key):
        old_ts = exp.index
        data = exp.as_matrix()
        data_ds = custom.resample(data, old_ts, ts, method=np.nanmean)
        epoch_df = pd.DataFrame(data_ds, columns=exp.columns, index=ts)
        epoch_df.index = pd.MultiIndex.from_product([[epoch_name], epoch_df.index], names=['epoch', 'timestamp'])
        subj_df = pd.concat([subj_df, epoch_df], axis=0)

    subj_df.columns = pd.MultiIndex.from_product([[subj], subj_df.columns], names=['subject', 'feature'])
    dfs.append(subj_df)

behav_df = pd.concat(dfs, axis=1, names=['subject', 'feature'])
behav_df = behav_df.sort_index(axis=1, level=0)

### Import calcium imaging data
Each session is 1499 or 1500 frames

In [ ]:
# Parameters

frame_dur = 200
epochs = ['baseline', 'h2o', 'tmt']

In [ ]:
# Import data

# Gather files
trace_files = glob.glob(trace_source)
event_files = glob.glob(event_source)

# Import data
trace_import = {
    os.path.basename(f).split('_')[1]: np.loadtxt(f, delimiter=',')
    for f in trace_files
}
event_import = {
    os.path.basename(f).split('_')[1]: np.loadtxt(f, delimiter=',')
    for f in event_files
}

In [ ]:
# Create dataframes for traces and events

subjs = trace_import.keys()
trace_df = pd.DataFrame()
event_df = pd.DataFrame()

for subj in subjs:
    img_ts = np.arange(trace_import[subj].T.shape[0]) * frame_dur
    ts_long = np.arange(0, img_ts[-1], bin_size)
    
    subj_index = pd.MultiIndex.from_product([epochs, ts], names=['epoch', 'timestamps'])[:len(img_ts)]
    
#     trace_ds = custom.resample(trace_import[subj].T, img_ts, ts_long, method=np.nanmean)
#     event_ds = custom.resample(event_import[subj].T, img_ts, ts_long, method=np.nanmean)
#     subj_trace_df = pd.DataFrame(trace_ds, index=subj_index)
#     subj_event_df = pd.DataFrame(event_ds, index=subj_index)
    
    subj_trace_df = pd.DataFrame(trace_import[subj].T, index=subj_index)
    subj_event_df = pd.DataFrame(event_import[subj].T, index=subj_index)
    
    subj_trace_df.columns = pd.MultiIndex.from_product([[subj], subj_trace_df.columns])
    subj_event_df.columns = pd.MultiIndex.from_product([[subj], subj_event_df.columns])
    
    trace_df = pd.concat([trace_df, subj_trace_df], axis=1)
    event_df = pd.concat([event_df, subj_event_df], axis=1)

neural_df = pd.concat([trace_df, event_df], axis=1, keys=['trace', 'event'], names=['datatype', 'subject', 'neuron'])
neural_df = neural_df.sort_index(axis=1, level=0)

In [ ]:
# Check

subj = np.random.choice(trace_import.keys())
cell = np.random.choice(np.arange(trace_import[subj].shape[0]))

fig, ax = plt.subplots(ncols=3, figsize=(15, 5))
ax[0].plot(trace_import[subj][cell]);
ax[1].plot(neural_df[('trace', subj, cell)].as_matrix());
ax[2].plot(trace_import[subj][cell]);
ax[2].plot(neural_df[('trace', subj, cell)].as_matrix());

## Save data

In [ ]:
with pd.HDFStore(h5_outfile) as hf:
    hf['behav'] = behav_df
    hf['neural'] = neural_df

---
<a id='epm'></a>
# Elevated-plus maze

In [ ]:
behav_source = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/PNOC_EPM/PNOC_Behavior/*.xlsx'
trace_source = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/PNOC_EPM/PNOC_Traces/*.txt'
event_source = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/PNOC_EPM/PNOC_Spikes/*.txt'

h5_outfile = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/epm-200msbin.h5'

## Import data

In [ ]:
# Parameters

bin_size = 200
exp_dur = 600000
n_cores = 8

### Import behavioral data

In [ ]:
# Define import function

def import_behav(filename):
    subj, epoch = os.path.splitext(os.path.basename(filename))[0].split('_')
    data = custom.etho_extract(filename)
    data.index = data.index * 1000
    
    return (subj, epoch), data

In [ ]:
# Import data

behav_files = glob.glob(behav_source)
p = mp.Pool(processes=n_cores)
exps, behav_import = zip(*p.map(import_behav, behav_files))

In [ ]:
# Correct for 10-s extra at beginning of behavioral data
for data in behav_import:
    data.index -= 10000

### Downsample data

In [ ]:
# Create dataframe from all animals

ts = np.arange(0, exp_dur, bin_size)
subjs = [x for x, _ in exps]

dfs = {}  # Dictionary to store DataFrame from each animal
for subj, data in zip(subjs, behav_import):
    data_ds = custom.resample(data, data.index, ts, method=np.nanmean)
    ds_df = pd.DataFrame(data_ds, columns=data.columns, index=ts)
    ds_df.columns.names = ['feature']
    ds_df.index.names = ['timestamp']
    dfs[subj] = ds_df

# Create DataFrame for all data
behav_df = pd.concat(dfs, axis=1, names=['subject', 'feature'])
behav_df = behav_df.sort_index(axis=1, level=0)

### Import calcium imaging data
Each session is 1499 or 1500 frames

In [ ]:
# Parameters

frame_dur = 200

In [ ]:
# Gather files
trace_files = glob.glob(trace_source)
event_files = glob.glob(event_source)

# Import data
trace_import = {
    os.path.basename(f).split('_')[1]: pd.DataFrame(np.loadtxt(f, delimiter=',').T)
    for f in trace_files
}
event_import = {
    os.path.basename(f).split('_')[1]: pd.DataFrame(np.loadtxt(f, delimiter=',').T)
    for f in event_files
}

In [ ]:
trace_import['PNOC188']

In [ ]:
# Create DataFrame

trace_df = pd.concat(trace_import, axis=1)
event_df = pd.concat(event_import, axis=1)
neural_df = pd.concat([trace_df, event_df], axis=1, keys=['trace', 'event'], names=['datatype', 'subject', 'neuron'])
neural_df.index = np.arange(0, exp_dur, frame_dur)

### Downsample data (if necessary)

In [ ]:
ts = np.arange(0, exp_dur, bin_size)
data_ds = custom.resample(neural_df_orig, neural_df.index, ts, method=np.nanmean)

In [ ]:
neural_df = pd.DataFrame(data_ds, columns=neural_df_orig.columns, index=ts)
neural_df.index.name = 'timestamp'

### Save data

In [ ]:
with pd.HDFStore(h5_outfile) as hf:
    hf['behav'] = behav_df
    hf['neural'] = neural_df

---
<a id='headfixed'></a>
# Headfixed exposure
Create behavioral file with `pupilize`

In [2]:
frame_dur = 200
threshold = 225

del_pnt = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/del_hf-pnt.csv'
del_tmt = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/del_hf-tmt.csv'

hf_match_file = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/matches.csv'

h5_out = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/headfixed.h5'

raw_data_pnt = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFPNT/PNOC_Behavior'
h5_out_pnt = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/hf-data-pnt.h5'

raw_data_tmt = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFTMT/PNOC_Behavior'
h5_out_tmt = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/hf-data-tmt.h5'

ca_files = glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFPNT/PNOC_Traces/*.txt') + \
           glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFTMT/PNOC_Traces/*.txt')# + \
#            glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFPNT/PNOC_Spikes/*.txt') + \
#            glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFTMT/PNOC_Spikes/*.txt')

# trace_files = glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFPNT/PNOC_Traces/*.txt') + \
#               glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFTMT/PNOC_Traces/*.txt')
# event_files = glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFPNT/PNOC_Spikes/*.txt') + \
#               glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFTMT/PNOC_Spikes/*.txt')

# trace_files_pnt = glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFPNT/PNOC_Traces/*.txt')
# event_files_pnt = glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFPNT/PNOC_Spikes/*.txt')
# h5_out_pnt = '/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/hf-data-pnt.h5'

# trace_files_tmt = glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFTMT/PNOC_Traces/*.txt')
# event_files_tmt = glob.glob('/data/Dropbox (Stuber Lab)/We PNOC-ing/Latest PNOC Data/2P Data/PNOC_HFTMT/PNOC_Spikes/*.txt')

## Create behavioral data

In [24]:
%%bash -s "$h5_out_pnt" "$raw_data_pnt" "$frame_dur" "$threshold"

"/data/Dropbox (Stuber Lab)/analysis/pnoc/organize_behav.py" -n 7 -t $4 -b $3 -o "$1" "$2"

Analyzing Behavior_J53_PNT_A_P1_3.h5...
Finished
Analyzing Behavior_J53_PNT_A_P2_3.h5...
Finished
Analyzing Behavior_J51_PNT_B_P2_2.h5...
Finished
Analyzing Behavior_J52_PNT_B_P1_3.h5...
Finished
Analyzing Behavior_J54_PNT_A_P1_1.h5...
Finished
Analyzing Behavior_J52_PNT_B_P2_3.h5...
Finished
Analyzing Behavior_J54_PNT_A_P2_3.h5...
Finished
Analyzing Behavior_J54_PNT_A_P1_3.h5...
Finished
Analyzing Behavior_J55_PNT_A_P1_1.h5...
Finished
Analyzing Behavior_J50_PNT_B_P1_3.h5...
Finished
Analyzing Behavior_J31_PNT_B_P1_2.h5...
Finished
Analyzing Behavior_J51_PNT_B_P2_3.h5...
Finished
Analyzing Behavior_J54_PNT_A_P2_1.h5...
Finished
Analyzing Behavior_J54_PNT_A_P2_2.h5...
Finished
Analyzing Behavior_J52_PNT_B_P1_1.h5...
Finished
Analyzing Behavior_J31_PNT_B_P1_1.h5...
Finished
Analyzing Behavior_J55_PNT_A_P1_2.h5...
Finished
Analyzing Behavior_J50_PNT_B_P1_1.h5...
Finished
Analyzing Behavior_J52_PNT_B_P1_2.h5...
Finished
Analyzing Behavior_J51_PNT_B_P1_1.h5...
Finished
Analyzing Behavior_J

In [25]:
%%bash -s "$h5_out_tmt" "$raw_data_tmt" "$frame_dur" "$threshold"

"/data/Dropbox (Stuber Lab)/analysis/pnoc/organize_behav.py" -n 7 -t $4 -b $3 -o "$1" "$2"

Analyzing Behavior_J52_TMT_A_P1_3.h5...
Finished
Analyzing Behavior_J31_TMT_A_P1_2.h5...
Finished
Analyzing Behavior_J53_TMT_B_P2_3.h5...
Finished
Analyzing Behavior_J52_TMT_A_P1_2.h5...
Finished
Analyzing Behavior_J51_TMT_A_P2_3.h5...
Finished
Analyzing Behavior_J51_TMT_A_P1_1.h5...
Finished
Analyzing Behavior_J52_TMT_A_P2_1.h5...
Finished
Analyzing Behavior_J53_TMT_B_P2_2.h5...
Finished
Analyzing Behavior_J53_TMT_B_P1_3.h5...
Finished
Analyzing Behavior_J55_TMT_B_P1_3.h5...
Finished
Analyzing Behavior_J53_TMT_B_P1_2.h5...
Finished
Analyzing Behavior_J52_TMT_A_P2_2.h5...
Finished
Analyzing Behavior_J51_TMT_A_P1_3.h5...
Finished
Analyzing Behavior_J31_TMT_A_P1_1.h5...
Finished
Analyzing Behavior_J55_TMT_B_P1_1.h5...
Finished
Analyzing Behavior_J55_TMT_B_P1_2.h5...
Finished
Analyzing Behavior_J52_TMT_A_P2_3.h5...
Finished
Analyzing Behavior_J51_TMT_A_P2_2.h5...
Finished
Analyzing Behavior_J50_TMT_A_P1_1.h5...
Finished
Analyzing Behavior_J31_TMT_A_P1_3.h5...
Finished
Analyzing Behavior_J

In [26]:
# Combine datasets

# Read individual dataset files
with pd.HDFStore(h5_out_pnt, 'r') as hf_pnt, pd.HDFStore(h5_out_tmt, 'r') as hf_tmt:
    df_behav = pd.concat(
        [hf_pnt['behav'], hf_tmt['behav']],
        axis=1,
        keys=['PNT', 'TMT'],
        names=['experiment', ] + hf_pnt['behav'].columns.names
    )

# Rename index
df_behav = df_behav.rename(index={'ctrl': 'h2o', 'stim': 'odor'})

# Combine datasets
with pd.HDFStore(h5_out) as hf:
    hf['behav'] = df_behav

# Remove individual dataset files
# os.remove(h5_out_pnt)
# os.remove(h5_out_tmt)

## Import neural data

In [27]:
# Parameters

frames_per_epoch = 1505
frame_period = 200

Need to check frame counts on new files

In [28]:
# Number of frames for each epoch
frame_ct = {
    'J31_TMT_A_P1': [1505, 1505, 1505],
    'J50_TMT_A_P1': [1505, 1505, 1505],
    'J51_TMT_A_P1': [1504, 1504, 1504],
    'J51_TMT_A_P2': [1504, 1504, 1504],
    'J52_TMT_A_P1': [1505, 1505, 1505],
    'J52_TMT_A_P2': [1505, 1505, 1504],
    'J53_TMT_B_P1': [1505, 1505, 1505],
    'J53_TMT_B_P2': [1505, 1505, 1505],
    'J54_PNT_B_P1': [1505, 1505, 1505], #
    'J54_PNT_B_P2': [1505, 1505, 1505], #
    'J55_TMT_B_P1': [1505, 1505, 1505],
    'J31_PNT_B_P1': [1505, 1505, 1505],
    'J50_PNT_B_P1': [1505, 1505, 1505],
    'J51_PNT_B_P1': [1505, 1505, 1505],
    'J51_PNT_B_P2': [1505, 1505, 1505],
    'J52_PNT_B_P1': [1505, 1505, 1505],
    'J52_PNT_B_P2': [1505, 1505, 1505],
    'J53_PNT_A_P1': [1505, 1505, 1505],
    'J53_PNT_A_P2': [1505, 1505, 1504],
    'J54_PNT_A_P1': [1505, 1505, 1505], #
    'J54_PNT_A_P2': [1505, 1505, 1505], #
    'J55_PNT_A_P1': [1505, 1505, 1505],
}

In [29]:
# Import data

ca_import = {
    tuple(os.path.splitext(os.path.basename(f))[0].split('_')): np.loadtxt(f, delimiter=',')
    for f in ca_files
}

In [30]:
# Create dataframe

# Create new dictionary with key for each neuron
ca_traces = {}
for exp, traces in ca_import.iteritems():
    n_cells, n_frames = traces.shape
    exp_id = frame_ct['_'.join(exp[1:])]
    
    epoch_split = np.split(traces, np.cumsum(exp_id)[:2], axis=1)
    epoch_split_new = [
        np.concatenate([epoch, np.nan * np.zeros((n_cells, frames_per_epoch - nf))], axis=1)
        for epoch, nf in zip(epoch_split, exp_id)
    ]
    traces_new = np.concatenate(epoch_split_new, axis=1)
    
    for n, cell_trace in enumerate(traces_new):
        ca_traces[exp + (n, )] = cell_trace

# Create dataframe
neural_df = pd.DataFrame(ca_traces)

# Format columns
neural_df.columns.names = ['data type', 'subject', 'experiment', 'order', 'plane', 'neuron']
neural_df = neural_df.reorder_levels(['data type', 'experiment', 'subject', 'plane', 'order', 'neuron'], axis=1)
neural_df = neural_df.sort_index(axis=1)

# Format index
neural_df.index = pd.MultiIndex.from_product(
    [['base', 'h2o', 'odor'], np.arange(frames_per_epoch) * frame_period],
    names=['epoch', 'time']
)

In [31]:
# Remove bad data

delete_import = pd.concat(
    [pd.read_csv(del_pnt, delimiter=','), pd.read_csv(del_tmt, delimiter=',')],
    axis=1, keys=['PNT', 'TMT']
)
delete_import.columns = pd.MultiIndex.from_tuples(
    [[x[0], ] + ['PNOC' + x[1].split('_')[0]] + [x[1].split('_')[1]] for x in delete_import.columns]
)

# Cells to delete from TMT dataset
to_delete = [
    ('Traces', ) + col + (int(x) - 1, )
    for col in delete_import for x in delete_import[col]
    if not np.isnan(x)
]

neural_df = neural_df.drop(to_delete, axis=1, errors='ignore')
neural_df = neural_df.dropna(axis=0)

In [32]:
# Match cells
# Iterate over experiments, choosing only cells in peanut oil ones. Assign number based on match file.

match_import = pd.read_csv(hf_match_file, delimiter=',', header=0)
match_import -= 1  # start indexing at 0
match_import.columns = pd.MultiIndex.from_tuples(
    [[x.split('_')[0], x.split('_')[1]] for x in match_import.columns],
    names=['subject', 'plane']
)

neural_df_new = []  # new dataframe with updated matched cell labelling
for grp, grp_df in neural_df.groupby(level=['subject', 'plane'], axis=1):
    try:
        n_tmt_cells = max([x[-1] for x in grp_df.xs('TMT', axis=1, level='experiment')]) + 1
        print grp, n_tmt_cells
    except ValueError:
        print('No TMT experiments found for {}'.format(grp))
        continue
    key = match_import[grp]

    col = grp_df.columns.tolist()
    for n, x in enumerate(col):
        if 'PNT' in x:
            neuron_id = x[-1]
            matched_cell = key[neuron_id]
            if not np.isnan(matched_cell):
                col[n] = x[:-1] + (int(matched_cell), )
            else:
                col[n] = x[:-1] + (x[-1] + n_tmt_cells, )

    grp_df.columns = pd.MultiIndex.from_tuples(col, names=grp_df.columns.names)
    neural_df_new.append(grp_df)

neural_df_matched = pd.concat(neural_df_new, axis=1).sort_index(axis=1)

# # Following doesn't work...
# def match_cells(paired_df):
#     ''' Rename cells based on match from CSV file
#     Doesn't work because `groupby` won't change column names on original dataframe... -_-
#     '''
#     n_tmt_cells = max([x[-1] for x in paired_df.xs('TMT', axis=1, level='odor')]) + 1
#     key = match_import[paired_df.columns[0][2:4]]

#     col = paired_df.columns.tolist()
#     for n, x in enumerate(col):
#         if 'PNT' in x:
#             neuron_id = x[-1]
#             if not np.isnan(key[neuron_id]):
#                 col[n] = x[:-1] + (int(key[neuron_id]), )
#             else:
#                 col[n] = x[:-1] + (x[-1] + n_tmt_cells, )

#     paired_df.columns = pd.MultiIndex.from_tuples(col, names=paired_df.columns.names)
#     return paired_df

# neural_df = neural_df.groupby(level=['subject', 'plane'], axis=1).apply(match_cells)

('J31', 'P1') 64
('J50', 'P1') 38
('J51', 'P1') 120
('J51', 'P2') 93
('J52', 'P1') 179
('J52', 'P2') 70
('J53', 'P1') 238
('J53', 'P2') 168
No TMT experiments found for ('J54', 'P1')
No TMT experiments found for ('J54', 'P2')
('J55', 'P1') 76


### Save data

In [33]:
with pd.HDFStore(h5_out) as hf:
    hf['neural'] = neural_df_matched

In [34]:
# to_delete = [
#     ('Traces', 'TMT', 'PNOCJ51', 'P2', 3),
#     ('Traces', 'TMT', 'PNOCJ51', 'P2', 5),
#     ('Traces', 'TMT', 'PNOCJ51', 'P2', 9),
#     ('Traces', 'TMT', 'PNOCJ31', 'P1', 53),
#     ('Traces', 'TMT', 'PNOCJ31', 'P1', 60),
#     ('Traces', 'TMT', 'PNOCJ50', 'P1', 30),
#     ('Traces', 'TMT', 'PNOCJ50', 'P1', 37),
#     ('Traces', 'PNT', 'PNOCJ51', 'P1', 61),
#     ('Traces', 'PNT', 'PNOCJ52', 'P1', 40),
#     ('Traces', 'PNT', 'PNOCJ52', 'P1', 41),
#     ('Traces', 'PNT', 'PNOCJ52', 'P1', 73),
#     ('Traces', 'PNT', 'PNOCJ52', 'P2', 8),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 18),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 31),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 37),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 38),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 42),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 57),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 62),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 70),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 75),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 81),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 83),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 96),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 105),
#     ('Traces', 'PNT', 'PNOCJ31', 'P1', 129),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 24),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 33),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 37),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 38),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 39),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 42),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 46),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 47),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 50),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 51),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 52),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 53),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 55),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 56),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 57),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 58),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 60),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 61),
#     ('Traces', 'PNT', 'PNOCJ50', 'P1', 67),
# ]
